In [1]:
# ATSIT 2019: dimensional speech emotion recognition from text feature

# uncomment these to run on CPU only
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
import matplotlib.pyplot as plt
import pickle
import keras.backend as K
import pandas as pd
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, LSTM, TimeDistributed, Bidirectional, Embedding, Dropout, Flatten, concatenate, Convolution1D
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

from keras.callbacks import EarlyStopping
import codecs

import io
import random as rn
import tensorflow as tf

Using TensorFlow backend.
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/root/anaconda3/envs/ser36/lib/python3.6/sit

In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')

# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

# split train/test
split = 7869
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)



Found 3437 unique tokens
../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 128


In [4]:
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(32, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=True))(net)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=True))(net)
    net = Bidirectional(LSTM(32, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    #out_text = Dropout(0.5)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model
model = text_model1()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 300)     1031400     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 100, 64)      85248       embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 100, 128)     66048       bidirectional_1[0][0]            
____________________

In [5]:
# main function for for LOSO
def main(time):
    model = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model.fit( x_train_text[:8000], 
                      vad[:8000].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model.evaluate( x_train_text[8000:], vad[8000:].T.tolist())
    print(metrik[-3:])
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    weight = []
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('text_blstm_5layer.csv', header=['time','v','a','d', 'ave'])


Instructions for updating:
Use tf.cast instead.
Train on 6400 samples, validate on 1600 samples
Epoch 1/50
6400/6400 [==============================] - 89s 14ms/step - loss: 1.9403 - v_loss: 0.6226 - a_loss: 0.6566 - d_loss: 0.6610 - v_ccc: 0.3774 - a_ccc: 0.3434 - d_ccc: 0.3390 - val_loss: 2.0740 - val_v_loss: 0.7089 - val_a_loss: 0.6721 - val_d_loss: 0.6930 - val_v_ccc: 0.2911 - val_a_ccc: 0.3279 - val_d_ccc: 0.3070
Epoch 2/50
6400/6400 [==============================] - 83s 13ms/step - loss: 1.5142 - v_loss: 0.4209 - a_loss: 0.5483 - d_loss: 0.5450 - v_ccc: 0.5791 - a_ccc: 0.4517 - d_ccc: 0.4550 - val_loss: 2.0220 - val_v_loss: 0.6609 - val_a_loss: 0.6616 - val_d_loss: 0.6995 - val_v_ccc: 0.3391 - val_a_ccc: 0.3384 - val_d_ccc: 0.3005
Epoch 3/50
6400/6400 [==============================] - 85s 13ms/step - loss: 1.3481 - v_loss: 0.3557 - a_loss: 0.5039 - d_loss: 0.4886 - v_ccc: 0.6443 - a_ccc: 0.4961 - d_ccc: 0.5114 - val_loss: 1.8917 - val_v_loss: 0.6384 - val_a_loss: 0.6170 - val_d

Epoch 8/50
6400/6400 [==============================] - 89s 14ms/step - loss: 0.9199 - v_loss: 0.2211 - a_loss: 0.3511 - d_loss: 0.3477 - v_ccc: 0.7789 - a_ccc: 0.6489 - d_ccc: 0.6523 - val_loss: 1.8863 - val_v_loss: 0.5917 - val_a_loss: 0.6275 - val_d_loss: 0.6672 - val_v_ccc: 0.4083 - val_a_ccc: 0.3725 - val_d_ccc: 0.3328
Epoch 9/50
6400/6400 [==============================] - 89s 14ms/step - loss: 0.8629 - v_loss: 0.2069 - a_loss: 0.3292 - d_loss: 0.3268 - v_ccc: 0.7931 - a_ccc: 0.6708 - d_ccc: 0.6732 - val_loss: 1.9243 - val_v_loss: 0.6059 - val_a_loss: 0.6537 - val_d_loss: 0.6647 - val_v_ccc: 0.3941 - val_a_ccc: 0.3463 - val_d_ccc: 0.3353
Epoch 10/50
6400/6400 [==============================] - 89s 14ms/step - loss: 0.8175 - v_loss: 0.1977 - a_loss: 0.3097 - d_loss: 0.3101 - v_ccc: 0.8023 - a_ccc: 0.6903 - d_ccc: 0.6899 - val_loss: 1.9002 - val_v_loss: 0.5910 - val_a_loss: 0.6534 - val_d_loss: 0.6558 - val_v_ccc: 0.4090 - val_a_ccc: 0.3466 - val_d_ccc: 0.3442
Epoch 11/50
6400/6400

Epoch 11/50
6400/6400 [==============================] - 90s 14ms/step - loss: 0.7540 - v_loss: 0.1817 - a_loss: 0.2887 - d_loss: 0.2836 - v_ccc: 0.8183 - a_ccc: 0.7113 - d_ccc: 0.7164 - val_loss: 1.8973 - val_v_loss: 0.5768 - val_a_loss: 0.6371 - val_d_loss: 0.6834 - val_v_ccc: 0.4232 - val_a_ccc: 0.3629 - val_d_ccc: 0.3166
Epoch 12/50
6400/6400 [==============================] - 89s 14ms/step - loss: 0.7128 - v_loss: 0.1732 - a_loss: 0.2732 - d_loss: 0.2665 - v_ccc: 0.8268 - a_ccc: 0.7268 - d_ccc: 0.7335 - val_loss: 1.9148 - val_v_loss: 0.5864 - val_a_loss: 0.6377 - val_d_loss: 0.6907 - val_v_ccc: 0.4136 - val_a_ccc: 0.3623 - val_d_ccc: 0.3093
Epoch 13/50
6400/6400 [==============================] - 89s 14ms/step - loss: 0.6747 - v_loss: 0.1622 - a_loss: 0.2593 - d_loss: 0.2532 - v_ccc: 0.8378 - a_ccc: 0.7407 - d_ccc: 0.7468 - val_loss: 1.9489 - val_v_loss: 0.5843 - val_a_loss: 0.6579 - val_d_loss: 0.7066 - val_v_ccc: 0.4157 - val_a_ccc: 0.3421 - val_d_ccc: 0.2934
Epoch 14/50
6400/64

6400/6400 [==============================] - 89s 14ms/step - loss: 1.0604 - v_loss: 0.2651 - a_loss: 0.4042 - d_loss: 0.3911 - v_ccc: 0.7349 - a_ccc: 0.5958 - d_ccc: 0.6089 - val_loss: 1.9263 - val_v_loss: 0.6013 - val_a_loss: 0.6264 - val_d_loss: 0.6986 - val_v_ccc: 0.3987 - val_a_ccc: 0.3736 - val_d_ccc: 0.3014
Epoch 7/50
6400/6400 [==============================] - 89s 14ms/step - loss: 0.9824 - v_loss: 0.2465 - a_loss: 0.3744 - d_loss: 0.3615 - v_ccc: 0.7535 - a_ccc: 0.6256 - d_ccc: 0.6385 - val_loss: 1.8750 - val_v_loss: 0.5830 - val_a_loss: 0.6264 - val_d_loss: 0.6657 - val_v_ccc: 0.4170 - val_a_ccc: 0.3736 - val_d_ccc: 0.3343
Epoch 8/50
6400/6400 [==============================] - 89s 14ms/step - loss: 0.9167 - v_loss: 0.2277 - a_loss: 0.3503 - d_loss: 0.3388 - v_ccc: 0.7723 - a_ccc: 0.6497 - d_ccc: 0.6612 - val_loss: 1.9158 - val_v_loss: 0.5783 - val_a_loss: 0.6283 - val_d_loss: 0.7093 - val_v_ccc: 0.4217 - val_a_ccc: 0.3717 - val_d_ccc: 0.2907
Epoch 9/50
6400/6400 [===========

Epoch 14/50
6400/6400 [==============================] - 88s 14ms/step - loss: 0.6473 - v_loss: 0.1579 - a_loss: 0.2457 - d_loss: 0.2437 - v_ccc: 0.8421 - a_ccc: 0.7543 - d_ccc: 0.7563 - val_loss: 2.0005 - val_v_loss: 0.5853 - val_a_loss: 0.7214 - val_d_loss: 0.6938 - val_v_ccc: 0.4147 - val_a_ccc: 0.2786 - val_d_ccc: 0.3062
Epoch 15/50
6400/6400 [==============================] - 87s 14ms/step - loss: 0.6133 - v_loss: 0.1523 - a_loss: 0.2340 - d_loss: 0.2270 - v_ccc: 0.8477 - a_ccc: 0.7660 - d_ccc: 0.7730 - val_loss: 1.9376 - val_v_loss: 0.5872 - val_a_loss: 0.6609 - val_d_loss: 0.6895 - val_v_ccc: 0.4128 - val_a_ccc: 0.3391 - val_d_ccc: 0.3105
Epoch 16/50
6400/6400 [==============================] - 87s 14ms/step - loss: 0.5792 - v_loss: 0.1431 - a_loss: 0.2182 - d_loss: 0.2179 - v_ccc: 0.8569 - a_ccc: 0.7818 - d_ccc: 0.7821 - val_loss: 1.9248 - val_v_loss: 0.5807 - val_a_loss: 0.6681 - val_d_loss: 0.6760 - val_v_ccc: 0.4193 - val_a_ccc: 0.3319 - val_d_ccc: 0.3240
Epoch 17/50
2039/20

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 88s 14ms/step - loss: 0.6342 - v_loss: 0.1598 - a_loss: 0.2392 - d_loss: 0.2353 - v_ccc: 0.8402 - a_ccc: 0.7608 - d_ccc: 0.7647 - val_loss: 1.9270 - val_v_loss: 0.5955 - val_a_loss: 0.6537 - val_d_loss: 0.6778 - val_v_ccc: 0.4045 - val_a_ccc: 0.3463 - val_d_ccc: 0.3222
Epoch 15/50
6400/6400 [==============================] - 88s 14ms/step - loss: 0.6009 - v_loss: 0.1459 - a_loss: 0.2314 - d_loss: 0.2236 - v_ccc: 0.8541 - a_ccc: 0.7686 - d_ccc: 0.7764 - val_loss: 1.9180 - val_v_loss: 0.5721 - val_a_loss: 0.6569 - val_d_loss: 0.6889 - val_v_ccc: 0.4279 - val_a_ccc: 0.3431 - val_d_ccc: 0.3111
Epoch 16/50
6400/6400 [==============================] - 88s 14ms/step - loss: 0.5722 - v_loss: 0.1410 - a_loss: 0.2187 - d_loss: 0.2125 - v_ccc: 0.8590 - a_ccc: 0.7813 - d_ccc: 0.7875 - val_loss: 1.9562 - val_v_loss: 0.5912 - val_a_loss: 0.6675 - val_d_loss: 0.6976 - val_v_ccc: 0.4088 - val_a_ccc: 0.3325 - val_d_ccc: 0.3024
Epoch 17/50
6400/6400 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6400/6400 [==============================] - 89s 14ms/step - loss: 0.5304 - v_loss: 0.1349 - a_loss: 0.2003 - d_loss: 0.1953 - v_ccc: 0.8651 - a_ccc: 0.7997 - d_ccc: 0.8047 - val_loss: 1.9382 - val_v_loss: 0.5863 - val_a_loss: 0.6511 - val_d_loss: 0.7009 - val_v_ccc: 0.4137 - val_a_ccc: 0.3489 - val_d_ccc: 0.2991
Epoch 19/50
6400/6400 [==============================] - 88s 14ms/step - loss: 0.5089 - v_loss: 0.1259 - a_loss: 0.1937 - d_loss: 0.1893 - v_ccc: 0.8741 - a_ccc: 0.8063 - d_ccc: 0.8107 - val_loss: 1.9287 - val_v_loss: 0.5931 - val_a_loss: 0.6542 - val_d_loss: 0.6814 - val_v_ccc: 0.4069 - val_a_ccc: 0.3458 - val_d_ccc: 0.3186
Epoch 20/50
6400/6400 [==============================] - 88s 14ms/step - loss: 0.4871 - v_loss: 0.1250 - a_loss: 0.1826 - d_loss: 0.1796 - v_ccc: 0.8750 - a_ccc: 0.8174 - d_ccc: 0.8204 - val_loss: 1.9821 - val_v_loss: 0.5960 - val_a_loss: 0.6933 - val_d_loss: 0.6928 - val_v_ccc: 0.4040 - val_a_ccc: 0.3067 - val_d_ccc: 0.3072
Epoch 21/50
6400/6400 [========

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

